# Week 3 - Binary Classification (Logistic Regression)

## 3.1 Churn Prediction
Imagine you work at a company Telco and the company has tasked you to predict the churn of customers.

## 3.2 Data preparation
- download the data, read it with pandas
- look at the data
- make column names and values look uniform
- check if all the columns read correctly
- check if the churn variable needs any preparation

In [ ]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv'